<a href="https://colab.research.google.com/github/greygrease/tf-stuff/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdunction to NLP in Tensorlfow

NLP has the goal of deriving information our of natural language (text or speech)

Commom NLP problems are sequence to sequence problems (seq2seq)

## Get helper function

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-09-12 11:21:42--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-12 11:21:42 (67.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# Import series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback,compare_historys, plot_loss_curves

## Get a text dataset
Kaggle's Introdunction to NLP dataset



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-09-12 11:21:45--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 142.250.99.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-09-12 11:21:46 (150 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
# Unzip data

unzip_data('nlp_getting_started.zip')

## Visualize a text dataset

In [5]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [6]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
len(train_df)

7613

In [11]:
len(test_df)

3263

In [12]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text', 'target']][random_index : random_index + 5].itertuples():
  print(row)
  _, text, target = row
  print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n,{text}\n')
  print(f'----\n')

Pandas(Index=792, text='City program to help turn blight into greenspace: The Tennessee Housing Development\x89Û_ http://t.co/ZZcbBQyJ1q  #news http://t.co/KKSgHsblFH', target=0)
Target: 0 (not real disaster)
Text:
,City program to help turn blight into greenspace: The Tennessee Housing DevelopmentÛ_ http://t.co/ZZcbBQyJ1q  #news http://t.co/KKSgHsblFH

----

Pandas(Index=6262, text='omfg.... I just woke up again....', target=0)
Target: 0 (not real disaster)
Text:
,omfg.... I just woke up again....

----

Pandas(Index=1855, text="I'm my own woman crush ????", target=0)
Target: 0 (not real disaster)
Text:
,I'm my own woman crush ????

----

Pandas(Index=1980, text='#Rohingya houses in #Kyee NockThie hamlet from Taungbazar region in Buthidaung were severely damaged in Cyclone @KasitaRoch @VivianUNHCR', target=1)
Target: 1 (real disaster)
Text:
,#Rohingya houses in #Kyee NockThie hamlet from Taungbazar region in Buthidaung were severely damaged in Cyclone @KasitaRoch @VivianUNHCR

----



### Split data into training and validation sets


In [14]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                              train_df_shuffled['target'].to_numpy(),
                                                              test_size = 0.1,
                                                              random_state = 42)

In [15]:
# Check the lenghts

len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Convert text into numbers

Tokenisation - Direct mapping of tokens

Embedding - Create a matrix of feature vectors


### Text vectorisation

In [16]:
train_sentences[:3]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....'],
      dtype=object)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None, # limit on the no of tokens (uncommon words get replaced by <OOV>)
                                     standardize = 'lower_and_strip_punctuation', # lowers text and removes punctuation
                                     split = 'whitespace', #split where the white spaces exist
                                     ngrams = None, 
                                     output_mode = 'int',
                                    output_sequence_length = None, # how long the sequences should be
                                    # pad_to_max_tokens = True
                                    )

In [18]:
# Find the average number of tokens in thee trainint twee
ln = (sum([len(i.split()) for i in train_sentences]))/len(train_sentences)
round(ln)

15

In [19]:
# Setup text vectorisation variables

max_vocab_length = 10000
max_length = round(ln)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [20]:
# Fit the text vectoriser

text_vectorizer.adapt(train_sentences)

In [21]:
sample_sentence = "There's a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Choose a random sentence from the training dataset and tokenize it 

random_sentence = random.choice(train_sentences)
print(f'OG sent:\n {random_sentence}\n')

print(f'Vectorized sent:\n {text_vectorizer([random_sentence])}')

OG sent:
 Warne Ponting shocked by Australian collapse - Yahoo Cricket India https://t.co/hsgkTeZUCN

Vectorized sent:
 [[3262 9687 3421   18 2189  155 3237 3963  544    1    0    0    0    0
     0]]


In [23]:
text_vectorizer([random_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[3262, 9687, 3421,   18, 2189,  155, 3237, 3963,  544,    1,    0,
           0,    0,    0,    0]])>

In [24]:
# Get the unique words in our vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()

words_in_vocab[:5]


['', '[UNK]', 'the', 'a', 'in']

In [25]:
words_in_vocab[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [26]:
len(words_in_vocab)

10000

In [27]:
words_in_vocab[-1]

'paddytomlinson1'

### Create an embedding representation of words

Parameters:

      input_dim = size of vocab
      output_dim = size of the output_sequence_length
      input_length = length of the seq


In [28]:
tf.random.set_seed(42)
from tensorflow.keras import layers

vocab_length = len(words_in_vocab)

embedding = layers.Embedding(input_dim = vocab_length,
                             output_dim = 128,
                             embeddings_initializer = 'uniform',
                             input_length = max_length)

In [29]:
# Get a random sentence from the training set

random_sentence = random.choice(train_sentences)

print(f'Original text:\n {random_sentence}\n')

print('After Embedding:')

# Embed the ranodm sent

sample_embed = embedding(text_vectorizer([random_sentence]))

print(sample_embed)

Original text:
 *books a flight to Burma when country is enduring political unrest and a natural disaster* no wonder it was so cheap ay

After Embedding:
tf.Tensor(
[[[-4.06648144e-02  3.00095119e-02 -3.53840813e-02 ... -1.44617669e-02
   -4.88357805e-02  1.19285099e-02]
  [-4.28401344e-02 -1.48979798e-02 -1.59495957e-02 ... -1.16610639e-02
    3.06106247e-02  1.97214819e-02]
  [-4.31834571e-02 -1.28522515e-05 -1.20935664e-02 ... -1.54436752e-03
   -4.08354029e-02 -1.98509339e-02]
  ...
  [-4.28401344e-02 -1.48979798e-02 -1.59495957e-02 ... -1.16610639e-02
    3.06106247e-02  1.97214819e-02]
  [-4.33639884e-02  1.44880302e-02 -1.60450824e-02 ...  1.35695599e-02
    7.07391649e-03 -1.15825050e-02]
  [-2.43819952e-02  7.07149506e-04  2.69013643e-03 ...  1.25561468e-02
   -3.07368282e-02 -3.10621392e-02]]], shape=(1, 15, 128), dtype=float32)


In [30]:
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-4.06648144e-02,  3.00095119e-02, -3.53840813e-02, ...,
         -1.44617669e-02, -4.88357805e-02,  1.19285099e-02],
        [-4.28401344e-02, -1.48979798e-02, -1.59495957e-02, ...,
         -1.16610639e-02,  3.06106247e-02,  1.97214819e-02],
        [-4.31834571e-02, -1.28522515e-05, -1.20935664e-02, ...,
         -1.54436752e-03, -4.08354029e-02, -1.98509339e-02],
        ...,
        [-4.28401344e-02, -1.48979798e-02, -1.59495957e-02, ...,
         -1.16610639e-02,  3.06106247e-02,  1.97214819e-02],
        [-4.33639884e-02,  1.44880302e-02, -1.60450824e-02, ...,
          1.35695599e-02,  7.07391649e-03, -1.15825050e-02],
        [-2.43819952e-02,  7.07149506e-04,  2.69013643e-03, ...,
          1.25561468e-02, -3.07368282e-02, -3.10621392e-02]]],
      dtype=float32)>

In [31]:
# Check out a singe token's embedding

sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.04066481,  0.03000951, -0.03538408,  0.02683998,  0.00321645,
        0.02805407, -0.00171066, -0.03023714, -0.00986227, -0.00593323,
        0.01556282,  0.00619644,  0.04483426, -0.01995292, -0.03300078,
        0.00091783, -0.01492354, -0.0125402 , -0.02580489,  0.00967506,
       -0.03928965, -0.04314787,  0.01657164,  0.01261712,  0.00775647,
       -0.00248822, -0.04490674,  0.00665697,  0.03294792,  0.03340543,
       -0.00100894,  0.01281964,  0.03765922,  0.00715588,  0.04988347,
       -0.00718862, -0.03291397,  0.01994849, -0.03451483,  0.02021727,
       -0.01357345, -0.04280755, -0.01344074,  0.01381474,  0.04063432,
       -0.03848456,  0.00105501,  0.03657391, -0.03993315,  0.02198092,
       -0.01312695,  0.04293266,  0.00125053, -0.02905765, -0.03710711,
       -0.00121229, -0.02954653,  0.04625065, -0.02122564,  0.01188238,
        0.00371226,  0.01586843,  0.04487607, -0.03098633,  0.01367304,
        0.040587

In [32]:
random_sentence

'*books a flight to Burma when country is enduring political unrest and a natural disaster* no wonder it was so cheap ay'

## Modelling a text dataet and running a series of experiments

    Model 0 : Naive Bayes (baseline)

    Model 1 : Feed Forward Neural Network (Dense)

    Model 2 : LSTM model (RNN)

    Model 3 : GRU model (RNN)

    Model 4 : Bidirectional LSTM model (RNN)

    Model 5 : 1D CNN

    Model 6:  Tensorflow Hub pretrained Feature Extractor (transfer learning)

    Model 7: Same as model 6 with 10% of training data

### Model 0 : Getting a baseline 


As with all machine learning experiments its important to create a baseline model for benchmarking

> Its common practice to use non_DL as baseline and use DL to see if we can improve upon it

In [33]:
from tensorflow.python.ops.gen_random_ops import Multinomial
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # turn our text to numbers
    ('clf', MultinomialNB())     # model the text
])

# Fit the pipe line to the training ata

model_0.fit(train_sentences, train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [34]:
# Evaluate our baseline our model

baseline_score = model_0.score(val_sentences, val_labels)

In [35]:
print(f'Our baseline model achieves an accurayc of:{baseline_score*100}%')

Our baseline model achieves an accurayc of:79.26509186351706%


In [36]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [37]:
# Make predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

Compare our model using the following metrics:

>Accuracy

>Precision

>Recall

>F1-Score

In [38]:
# Function to eval 

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):

  # Accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Precison, Recall, F1-Score
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {'accuracy': model_accuracy,
                   'precision' : model_precision,
                   'recall': model_recall,
                   'f1': model_f1}
  return model_results

In [39]:
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1 : A simple dense model

In [40]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Createa a directory to save TensorBoard lofs
SAVE_DIR = 'model_logs'

In [41]:
# Build model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype = tf.string) # inputs are 1 dimensinal strting
x = text_vectorizer(inputs) # nums to vectors
x = embedding(x) 
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')


In [42]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [43]:
# Compile the model

model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [44]:
# Fit the model

model_history = model_1.fit(x = train_sentences,
                          y = train_labels,
                          epochs = 5,
                          validation_data = (val_sentences, val_labels),
                          callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                   experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220912-112153
Epoch 1/5
215/215 [==============================] - 8s 10ms/step - loss: 0.6094 - accuracy: 0.6916 - val_loss: 0.5357 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.4410 - accuracy: 0.8189 - val_loss: 0.4691 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3463 - accuracy: 0.8605 - val_loss: 0.4590 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2848 - accuracy: 0.8923 - val_loss: 0.4641 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2380 - accuracy: 0.9118 - val_loss: 0.4767 - val_accuracy: 0.7874


In [45]:
# Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4767 - accuracy: 0.7874


[0.47668465971946716, 0.787401556968689]

In [46]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0]

array([0.40488204], dtype=float32)

In [47]:
model_1_pred_probs.shape

(762, 1)

In [48]:
# Convert model prediction probs to label formats

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [49]:
# Calcualte model 1 results
model_1_results = calculate_results(y_true = val_labels,
                            y_pred = model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

## Visualizing learned embeddings



In [50]:
# Get the vocab from the text vectorization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [51]:
words_in_vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [52]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [53]:
# Get the wieght matrix of embedding layer

# These are the numerical representation of each token in our training data which have been 

embed_weights = model_1.get_layer('embedding').get_weights()[0]
embed_weights

array([[ 0.00073164,  0.01504799, -0.03425455, ..., -0.0440354 ,
        -0.01042279,  0.01876439],
       [ 0.04135865, -0.03945084, -0.03811941, ...,  0.00464736,
         0.03163552,  0.029283  ],
       [ 0.0068403 ,  0.05363134, -0.00241555, ..., -0.07082177,
        -0.04750707,  0.01448254],
       ...,
       [-0.03301444, -0.0052493 , -0.04209725, ...,  0.02028764,
         0.00308807,  0.02215792],
       [ 0.00692343,  0.05942352, -0.01975194, ..., -0.06199061,
        -0.01018393,  0.03510419],
       [-0.0372346 ,  0.06267186, -0.07451145, ..., -0.02367217,
        -0.0864333 ,  0.01742155]], dtype=float32)

In [54]:
# Create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [55]:
# Download files from colab and upload to projector tool

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent neural network (RNN's)

> RNN's are useful for sequence data.

> The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of the later input.

> 

In [56]:
2784 *3*4

33408

### Model 2 : LSTM

LSTM = Long Short Term 

Structur of an RNN

    Input(text) -> Tokenize -> Embedding -> Layers(RNNs/Dense) -> Output (label probability)

In [57]:
# Create an LSTM Modle

from tensorflow.keras import layers

inputs = layers.Input(shape=(1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.LSTM(64, return_sequences = True)(x) # when you're stacking RNN cells together you need to return sequences
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
x = layers.Dense(64, activation = 'relu')(x)
# print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_LSTM')

In [58]:
# Get a summary

model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                      

In [59]:
# Compile the model
model_2.compile(loss= 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics =['accuracy'])

In [60]:
# Fit the model

model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'model_2')])

Saving TensorBoard log files to: model_logs/model_2/20220912-112217
Epoch 1/5
215/215 [==============================] - 8s 12ms/step - loss: 0.2189 - accuracy: 0.9250 - val_loss: 0.5444 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1634 - accuracy: 0.9412 - val_loss: 0.6071 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1282 - accuracy: 0.9520 - val_loss: 0.7574 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1076 - accuracy: 0.9584 - val_loss: 0.9243 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0815 - accuracy: 0.9635 - val_loss: 0.9809 - val_accuracy: 0.7808


In [61]:
model_2_pred_prob = model_2.predict(val_sentences)

In [62]:
model_2_pred_prob[:10]

array([[2.3730542e-03],
       [5.7842934e-01],
       [9.9988604e-01],
       [5.8599334e-02],
       [2.6671838e-05],
       [9.9545348e-01],
       [9.3895912e-01],
       [9.9989414e-01],
       [9.9978155e-01],
       [3.9918759e-01]], dtype=float32)

In [63]:
# Convert model 2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_prob))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [64]:
# Calcualte model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)

In [65]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [66]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [67]:
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

### Model_3: GRU

GRU - Gated Recurrent Unit
The GRU cell has similar features to an LSTM cell but has less parameters.

In [68]:
from tensorflow.keras import layers
inputs = layers.Input(shape =(1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x = layers.LSTM(64, return_sequences = True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name = 'model_3_GRU')

In [69]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [70]:
# Xomile the modle

model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics= ['accuracy'])

# Fit the model

model_3.history = model_3.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'model_3_Gru')])

Saving TensorBoard log files to: model_logs/model_3_Gru/20220912-112241
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.1620 - accuracy: 0.9377 - val_loss: 0.6392 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0887 - accuracy: 0.9672 - val_loss: 0.7860 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0717 - accuracy: 0.9721 - val_loss: 1.0992 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0630 - accuracy: 0.9739 - val_loss: 1.0155 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0519 - accuracy: 0.9758 - val_loss: 1.2545 - val_accuracy: 0.7782


In [71]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[4.7421403e-04],
       [7.2494876e-01],
       [9.9988055e-01],
       [9.6555941e-02],
       [6.1819854e-05],
       [9.9976522e-01],
       [9.6143895e-01],
       [9.9993181e-01],
       [9.9987459e-01],
       [9.8283893e-01]], dtype=float32)

In [72]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))

In [73]:
model_3_results = calculate_results(y_true = val_labels,
                                    y_pred = model_3_preds)

In [74]:
model_3_results

{'accuracy': 77.82152230971128,
 'precision': 0.7791991586469295,
 'recall': 0.7782152230971129,
 'f1': 0.7766208088282468}

In [75]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [76]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [77]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right to left.

Bidrectional RNN goes from right to left aswell as left to right

In [87]:
# Bulid a bidirectional RNN in tensorflow

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 =  tf.keras.Model(inputs, outputs, name = 'model_4_bidirectional')

In [88]:
# Model 4 summary

model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_8 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [89]:
# Compile the model

model_4.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])


In [91]:
# Fit the model

model_4_history = model_4.fit(train_sentences, train_labels, epochs  =5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'Model_4_Bidirectioanl')])

Saving TensorBoard log files to: model_logs/Model_4_Bidirectioanl/20220912-121557
Epoch 1/5
215/215 [==============================] - 6s 12ms/step - loss: 0.1075 - accuracy: 0.9692 - val_loss: 0.9485 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0528 - accuracy: 0.9761 - val_loss: 1.1411 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0482 - accuracy: 0.9787 - val_loss: 1.1724 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0415 - accuracy: 0.9803 - val_loss: 1.2619 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0371 - accuracy: 0.9826 - val_loss: 1.5939 - val_accuracy: 0.7703


In [94]:
# Make prediction 

model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[3.8419033e-03],
       [7.9843575e-01],
       [9.9998486e-01],
       [1.8698394e-01],
       [1.2805108e-05],
       [9.9994779e-01],
       [9.9174035e-01],
       [9.9999225e-01],
       [9.9998462e-01],
       [9.9847859e-01]], dtype=float32)

In [97]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [99]:
# Calculate the results of our bidirectional model

model_4_results = calculate_results(y_true = val_labels,
                                    y_pred = model_4_preds)

In [100]:
print( model_4_results, '\n', model_3_results, '\n', model_2_results, '\n', model_1_results, '\n', baseline_results)

{'accuracy': 77.03412073490814, 'precision': 0.7723473976820748, 'recall': 0.7703412073490814, 'f1': 0.7680604812955161} 
 {'accuracy': 77.82152230971128, 'precision': 0.7791991586469295, 'recall': 0.7782152230971129, 'f1': 0.7766208088282468} 
 {'accuracy': 78.08398950131233, 'precision': 0.7856875884556138, 'recall': 0.7808398950131233, 'f1': 0.777681907957685} 
 {'accuracy': 78.74015748031496, 'precision': 0.7914920592553047, 'recall': 0.7874015748031497, 'f1': 0.7846966492209201} 
 {'accuracy': 79.26509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


## Convolutional Neural Network for Text (and other type of sequences)

>Images are 2D (Conv2D)

>Text are 1D (Conv1D)

Typical Structure of Conv2D:

    Inputs(test) --> Tokenisatin --> Embedding --> Conv1D --> Pooling --> Outputs

### Model_5: Conv1D

In [103]:
# Test out our embedding layer, Conv1D layer and max pooling

embedding_text = embedding(text_vectorizer(['this is a test sentence'])) # Turn into Embedding
conv_1d = layers.Conv1D(filters = 32,
                        kernel_size = 5,
                        stride = 1,
                        activation = 'relu',
                        padding = 'valid')

In [115]:
print('TensorShape([No of Seq, Lenght of Sent, Dim of Feature Vector]')
embedding_test.shape

TensorShape([No of Seq, Lenght of Sent, Dim of Feature Vector]


TensorShape([1, 15, 128])

In [113]:
print('TensorShape([No of Seq, , No of hidden neurons]')
conv_1d_output.shape

TensorShape([1, 11, 32])

In [114]:
max_pool_output.shape

TensorShape([1, 32])

In [107]:
conv_1d_output = conv_1d(embedding_text)
conv_1d_output[0]

<tf.Tensor: shape=(11, 32), dtype=float32, numpy=
array([[0.00661569, 0.00726261, 0.01874273, 0.        , 0.03809398,
        0.        , 0.06253233, 0.        , 0.11049788, 0.        ,
        0.03504574, 0.02993291, 0.02437021, 0.        , 0.        ,
        0.        , 0.        , 0.0061935 , 0.03197248, 0.        ,
        0.        , 0.0635457 , 0.        , 0.        , 0.        ,
        0.        , 0.04733778, 0.01550949, 0.02332238, 0.00295364,
        0.0476075 , 0.03757278],
       [0.        , 0.        , 0.        , 0.        , 0.03149125,
        0.03985992, 0.        , 0.        , 0.        , 0.        ,
        0.0305372 , 0.03643144, 0.        , 0.05115056, 0.00254945,
        0.        , 0.01614198, 0.0539733 , 0.        , 0.03459722,
        0.        , 0.0559739 , 0.        , 0.02101539, 0.        ,
        0.00029052, 0.        , 0.04903356, 0.        , 0.03608448,
        0.        , 0.0864023 ],
       [0.03230791, 0.        , 0.        , 0.0087215 , 0.        ,


In [110]:
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # Get the most important feature
max_pool_output[0]

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0.03230791, 0.00726261, 0.04210207, 0.0087215 , 0.06461123,
       0.03985992, 0.06253233, 0.02080052, 0.11049788, 0.01849101,
       0.07309723, 0.03643144, 0.02437021, 0.05417022, 0.00254945,
       0.05296656, 0.01614198, 0.07019679, 0.03197248, 0.03459722,
       0.02196677, 0.0635457 , 0.05723992, 0.0842386 , 0.00372265,
       0.00029052, 0.04733778, 0.04903356, 0.03464232, 0.03608448,
       0.0476075 , 0.0864023 ], dtype=float32)>

In [109]:
embedding_text.shape

TensorShape([1, 15, 128])